In [1]:
from retrieve import retrieve
re = retrieve()


d:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load embedding model...  WARNING:tensorflow:From d:\Python\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

Done
Load vector store...  Done


In [2]:
from rerank import reranker
_reranker = reranker()

Load reranker model... Done


Đánh giá truy xuất trên 10 dữ liệu đầu tiên


In [ ]:
import json
import concurrent.futures

# Đọc dữ liệu từ file JSONL
with open("qs_context.jsonl", "r", encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# Hàm để rerank cho từng truy vấn
def rerank_for_query(query_data):
    query = query_data['Question']
    contexts = [j.page_content for j in re.similarity_search(query=query, top_k=10)]
    return _reranker.rerank(query=query, contexts=contexts)[0]

# Sử dụng ThreadPoolExecutor hoặc ProcessPoolExecutor để chạy song song
with concurrent.futures.ThreadPoolExecutor() as executor:
    retrieved_contexts = list(executor.map(rerank_for_query, data))

# retrieved_contexts sẽ chứa kết quả cho tất cả các truy vấn


In [ ]:
print(len(retrieved_contexts[1]))

10


In [ ]:
import numpy as np

def calculate_mrr(retrieved_contexts, relevant):
    """Tính MRR@10."""
    for rank, context in enumerate(retrieved_contexts[:10], 1):
        if context == relevant:  # context đúng là context duy nhất trong relevant
            return 1 / rank
    return 0  # Nếu không tìm thấy kết quả đúng trong 10 truy xuất

def calculate_recall(retrieved_contexts, relevant):
    """Tính Recall@10."""
    relevant_retrieved = [context for context in retrieved_contexts[:10] if context == relevant]
    return len(relevant_retrieved) / 1  # Vì chỉ có 1 context đúng trong relevant


# Khởi tạo các danh sách để lưu kết quả
mrr_scores = []
recall_scores = []

# Tính toán các chỉ số cho mỗi câu hỏi
for i in range(len(data)):
    retrieved = retrieved_contexts[i]
    relevant = data[i]['Context']
    
    # Tính MRR@10
    mrr_scores.append(calculate_mrr(retrieved, relevant))
    
    # Tính Recall@10
    recall_scores.append(calculate_recall(retrieved, relevant))

# Tính giá trị trung bình của các chỉ số
mean_mrr = np.mean(mrr_scores)
mean_recall = np.mean(recall_scores)

print(f"Mean MRR@10: {mean_mrr:.4f}")
print(f"Mean Recall@10: {mean_recall:.4f}")


Mean MRR@10: 0.7552
Mean Recall@10: 0.9168
